In [1]:
import cv2
import numpy as np
from PIL import Image
import threading
from threading import current_thread

In [2]:
movie_path = u"/home/alex/下载/学习/data/yttlj/23.mp4"
save_path = "/home/alex/下载/学习/data/23"

In [3]:
cap = cv2.VideoCapture(movie_path)

fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) / 2), 
        int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) / 2))

print('帧率：%.1f, 尺寸：(%d,%d)' %(fps,size[0], size[1]))
cv2.namedWindow("movie", cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)
cv2.resizeWindow("movie", size[0], size[1])

classfier = cv2.CascadeClassifier("/usr/local/lib/python3.6/dist-packages/cv2/data/haarcascade_frontalface_alt2.xml")

color = (0, 255, 0)

帧率：25.0, 尺寸：(960,540)


In [ ]:

flag = 1
count = 1
images = []
threads = []
done = True
lock = threading.Lock()


def main_process():
    print('start')
    global count
    global flag
    global images
    global threads
    global done
    while cap.isOpened():
        ok, v_show = cap.read()
        if not ok:
            print('done!')
            done = False
            break
    
        grey = cv2.cvtColor(v_show, cv2.COLOR_BGR2GRAY)
    
        faceRects = classfier.detectMultiScale(grey, scaleFactor=1.2, minNeighbors=3, minSize=(32,32))
        if len(faceRects) > 0:
            for faceRect in faceRects:
                x,y,w,h = faceRect
            
                img_name = '%s/%d.jpg'%(save_path, count)
                image = v_show[y-10:y+h+10, x-10:x+w+10]
                if flag%6 == 0:
                    # print('%d.jpg'%(count))
                    # resize_frame = cv2.resize(image, size, interpolation=cv2.INTER_AREA)
                    # cv2.imwrite(img_name,image)
                    img = { "name": img_name, "image": image }
                    images.append(img)
                    count += 1
            
                flag += 1
            
                cv2.rectangle(v_show, (x-10, y-10), (x+w+10, y+h+10), color, 2)
            
                # font = cv2.FONT_HERSHEY_SIMPLEX
                # cv2.putText(v_show, '%d' %(flag),(x+30,y+30), font, 1,(255,0,255),4)
            
    
        if flag%6 == 0:
            cv2.imshow("movie", v_show)
        k = cv2.waitKey(10) & 0xFF
        if k == ord('s'):
            print('w:%d, h:%d' %(cap.get(3), cap.get(4)))
        elif k == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    

def thread_process():
    global done
    while True:
        lock.acquire()
        if len(images) > 0:
            image = images.pop(0)
            cv2.imwrite(image['name'],image['image'])
            # print(current_thread().getName())
        if not done:
            break
        lock.release()
    if lock:
        lock.release()


def main():
    for i in range(3):
        t = threading.Thread(target=thread_process)
        threads.append(t)
        t.start()
    
    main_process()

    for i in range(3):
        threads[i].join()

if __name__ == '__main__':
    main()

start
